In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tqdm import tqdm
import warnings
import gc
warnings.filterwarnings("ignore")

2025-04-20 16:32:39.396173: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-20 16:32:39.547208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745191959.617865   19498 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745191959.638215   19498 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745191959.774320   19498 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
news_df = pd.read_table("./MINDsmall_train/news.tsv", header=None, names=[
    "news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"
])

behaviors_df = pd.read_table("./MINDsmall_train/behaviors.tsv", header=None, names=[
    "impression_id", "user_id", "time", "history", "impressions"
])


In [4]:
news_df.dropna(subset=['title'], inplace=True)
behaviors_df.dropna(subset=['impressions'], inplace=True)
print("News\n")
print(news_df.head())
print("Click behaviors")
print(behaviors_df.head())

News

  news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn

In [5]:
def extract_clicked_news(imp):
    return [i.split('-')[0] for i in imp.split() if i.endswith('-1')]

behaviors_df['clicked_news'] = behaviors_df['impressions'].apply(extract_clicked_news)

In [6]:
user_clicks = behaviors_df.explode('clicked_news')[['user_id', 'clicked_news']]
user_clicks.columns = ['user_id', 'news_id']
